# Pictopercept Image Pair Sampling

Creating balanced image pairs from Chicago Face Database for bias measurement.

In [7]:
# load necessary libraries and data

from skimpy import skim
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from itertools import combinations, product
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
np.random.seed(42)

# Define columns to keep - focused on earnings-relevant covariates
columns_to_keep = [
    'Model', 'EthnicitySelf', 'GenderSelf', 'AgeRated', 'FemaleProb', 'MaleProb', 
    'AsianProb', 'BlackProb', 'LatinoProb', 'MultiProb', 'OtherProb', 'WhiteProb', 
    'Attractive', 'Dominant', 'Trustworthy', 'LuminanceMedian'
]

df = pd.read_excel('../CFD 3.0 Norming Data and Codebook.xlsx', 
                   sheet_name='CFD U.S. Norming Data', 
                   header=7,
                   skiprows=[8],
                   usecols=columns_to_keep)

df

,Model,EthnicitySelf,GenderSelf,AgeRated,FemaleProb,MaleProb,AsianProb,BlackProb,LatinoProb,MultiProb,OtherProb,WhiteProb,Attractive,Dominant,Trustworthy,LuminanceMedian
0,AF-200,A,F,32.571429,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.111111,1.928571,3.925926,174.0
1,AF-201,A,F,23.666667,1.000000,0.000000,0.962963,0.000000,0.000000,0.037037,0.000000,0.000000,3.111111,2.111111,3.538462,172.0
2,AF-202,A,F,24.448276,0.827586,0.172414,0.310345,0.068966,0.137931,0.448276,0.034483,0.000000,3.000000,2.862069,3.379310,153.5
3,AF-203,A,F,22.758621,1.000000,0.000000,0.758621,0.000000,0.068966,0.172414,0.000000,0.000000,3.275862,1.750000,3.793103,175.5
4,AF-204,A,F,30.137931,1.000000,0.000000,0.827586,0.000000,0.068966,0.103448,0.000000,0.000000,3.172414,1.758621,3.310345,168.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,WM-254,W,M,30.307692,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.153846,3.230769,3.230769,174.0
593,WM-255,W,M,34.071429,0.035714,0.964286,0.000000,0.000000,0.111111,0.185185,0.111111,0.592593,2.259259,3.214286,2.892857,162.5
594,WM-256,W,M,30.961538,0.000000,1.000000,0.000000,0.000000,0.115385,0.076923,0.038462,0.769231,2.269231,3.230769,3.000000,166.5
595,WM-257,W,M,30.037037,0.000000,1.000000,0.000000,0.000000,0.111111,0.037037,0.000000,0.851852,3.740741,2.185185,3.923077,172.0


In [2]:
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 597    │ │ float64     │ 25    │                                                          │
│ │ Number of columns │ 28     │ │ string      │ 3     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column           ┃ NA  ┃ NA %  ┃ mean     ┃ sd       ┃ p0    ┃ p25   ┃ p50     ┃ p75     ┃ p100   ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩  │
│ │ AgeRated         │   0 │     0 │    28.86 │    6.299 │ 16.94 │ 24.65 │   27.28 │      32 │  56.38 │  ▃█▃▂  │  │
│ │ FemaleProb       │   0 │     0 │   0.5095 │   0.4918 │     0 │     0 │   0.913 │       1 │      1 │ █    █ │  │
│ │ MaleProb         │   0 │     0 │   0.4905 │   0.4918 │     0 │     0 │ 0.08696 │       1 │      1 │ █    █ │  │
│ │ AsianProb        │   0 │     0 │   0.1552 │   0.3142 │     0 │     0 │       0 │ 0.06897 │      1 │ █    ▁ │  │
│ │ BlackProb        │   0 │     0 │   0.2891 │   0.4272 │     0 │     0 │       0 │  0.8846 │      1 │ █    ▃ │  │
│ │ LatinoProb       │   0 │     0 │   0.1359 │   0.2248 │     0 │     0 │ 0.02247 │  0.1667 │      1 │  █▁▁▁  │  │
│ │ MultiProb        │   0 │     0 │  0.08178 │   0.1249 │     0 │     0 │ 0.03571 │  0.1111 │ 0.8966 │   █▁   │  │
│ │ OtherProb        │   0 │     0 │  0.02285 │  0.05016 │     0 │     0 │       0 │ 0.03448 │ 0.4074 │   █▁   │  │
│ │ WhiteProb        │   0 │     0 │   0.3152 │   0.4198 │     0 │     0 │ 0.01124 │    0.84 │      1 │ █    ▃ │  │
│ │ Afraid           │   0 │     0 │     1.92 │   0.3908 │  1.12 │ 1.652 │   1.872 │   2.124 │  3.899 │  ▃█▄▁  │  │
│ │ Angry            │   0 │     0 │    2.319 │   0.6247 │  1.16 │  1.88 │   2.208 │   2.699 │  4.815 │ ▄█▅▂▁  │  │
│ │ Attractive       │   0 │     0 │    3.232 │   0.7687 │  1.52 │ 2.688 │   3.144 │   3.731 │  5.478 │ ▂▆█▅▃▁ │  │
│ │ Babyfaced        │   0 │     0 │    2.396 │   0.6292 │  1.04 │ 1.962 │    2.32 │   2.802 │   4.37 │ ▂▇█▄▂▁ │  │
│ │ Disgusted        │   0 │     0 │    2.034 │    0.459 │ 1.143 │  1.72 │       2 │   2.321 │  3.753 │ ▃█▆▃▁  │  │
│ │ Dominant         │   0 │     0 │    2.832 │   0.6747 │ 1.333 │ 2.321 │   2.731 │   3.233 │  5.423 │ ▂█▇▃▁  │  │
│ │ Feminine         │   0 │     0 │     3.11 │    1.541 │     1 │   1.6 │   3.033 │   4.594 │      6 │ █▂▁▃▅▂ │  │
│ │ Happy            │   0 │     0 │     2.42 │   0.6438 │ 1.037 │ 1.926 │   2.321 │   2.778 │  4.542 │ ▂██▄▁▁ │  │
│ │ Masculine        │   0 │     0 │    3.271 │    1.415 │ 1.107 │ 1.885 │   3.226 │   4.621 │  6.192 │ █▆▂▅▇▂ │  │
│ │ Prototypic       │   0 │     0 │    3.263 │   0.8244 │ 1.022 │ 2.711 │   3.407 │   3.902 │  4.841 │ ▁▃▄▆█▃ │  │
│ │ Sad              │   0 │     0 │    2.419 │   0.5649 │ 1.269 │ 2.034 │    2.37 │    2.72 │    4.6 │ ▃█▇▃▁  │  │
│ │ Surprised        │   0 │     0 │    1.683 │   0.3082 │     1 │  1.48 │   1.654 │    1.84 │  4.367 │  ▅█▁   │  │
│ │ Threatening      │   0 │     0 │    2.154 │   0.5706 │ 1.103 │  1.75 │   2.065 │   2.464 │  4.385 │ ▄█▅▂▁  │  │
│ │ Trustworthy      │   0 │     0 │    3.429 │   0.4142

## Goal: 68 trials for Pictopercept study

- 64 target pairs testing gender/ethnicity bias in earnings perception
- 4 attention check pairs
- Focus on balancing age, attractiveness, dominance, trustworthiness

In [3]:
print(f"Original dataset shape: {df.shape}")
print(f"Unique ethnicities: {df['EthnicitySelf'].unique()}")
print(f"Unique genders: {df['GenderSelf'].unique()}")
print(f"\nDataset overview:")
print(df.groupby(['EthnicitySelf', 'GenderSelf']).size())

Original dataset shape: (597, 29)
Unique ethnicities: ['A' 'B' 'L' 'W']
Unique genders: ['F' 'M']

Dataset overview:
EthnicitySelf  GenderSelf
A              F              57
               M              52
B              F             104
               M              93
L              F              56
               M              52
W              F              90
               M              93
dtype: int64


## Data cleaning: keep faces where self-reports match rater perceptions

In [4]:
# Define earnings-relevant balancing covariates
balance_vars = ['AgeRated', 'Attractive', 'Trustworthy', 'Dominant']

# Check for missing data
print("Missing data check:")
missing_check = df[['Model', 'EthnicitySelf', 'GenderSelf'] + balance_vars].isnull().sum()
print(missing_check[missing_check > 0])

# Remove rows with missing values
df_clean = df.dropna(subset=['Model', 'EthnicitySelf', 'GenderSelf'] + balance_vars).copy()
print(f"\nAfter removing missing data: {df_clean.shape[0]} images")

Missing data check:
Series([], dtype: int64)

After removing missing data: 597 images


In [5]:
# Filter faces where category probabilities align with self-report
def filter_by_category_alignment(df, prob_threshold=0.60):
    # Gender alignment
    gender_aligned = (
        ((df['GenderSelf'] == 'F') & (df['FemaleProb'] > prob_threshold)) |
        ((df['GenderSelf'] == 'M') & (df['MaleProb'] > prob_threshold))
    )
    
    # Ethnicity alignment
    ethnicity_probs = df[['AsianProb', 'BlackProb', 'LatinoProb', 'WhiteProb', 'MultiProb', 'OtherProb']]
    ethnicity_argmax = ethnicity_probs.idxmax(axis=1)
    
    ethnicity_map = {'A': 'AsianProb', 'B': 'BlackProb', 'L': 'LatinoProb', 'W': 'WhiteProb'}
    expected_col = df['EthnicitySelf'].map(ethnicity_map)
    
    ethnicity_vals = []
    for idx, col in expected_col.items():
        if pd.notna(col):
            ethnicity_vals.append(df.loc[idx, col])
        else:
            ethnicity_vals.append(0)
    ethnicity_vals = pd.Series(ethnicity_vals, index=df.index)
    
    ethnicity_aligned = (
        (ethnicity_vals > prob_threshold) &
        (ethnicity_argmax == expected_col)
    )
    
    return df[gender_aligned & ethnicity_aligned].copy()

df_aligned = filter_by_category_alignment(df_clean, prob_threshold=0.60)
print(f"After category alignment filtering: {df_aligned.shape[0]} images")
print(f"Final sample by group:")
print(df_aligned.groupby(['EthnicitySelf', 'GenderSelf']).size())

After category alignment filtering: 466 images
Final sample by group:
EthnicitySelf  GenderSelf
A              F             46
               M             39
B              F             98
               M             80
L              F             22
               M             15
W              F             81
               M             85
dtype: int64


## Normalize earnings-relevant covariates and remove outliers

In [8]:
# Normalize balancing covariates to Z-scores
df_normalized = df_aligned.copy()

for var in balance_vars:
    df_normalized[f'{var}_z'] = (df_aligned[var] - df_aligned[var].mean()) / df_aligned[var].std()

balance_vars_z = [f'{var}_z' for var in balance_vars]

# Remove extreme outliers (|Z| > 3)
outlier_mask = np.abs(df_normalized[balance_vars_z]).max(axis=1) > 3
print(f"Found {outlier_mask.sum()} extreme outliers")

df_final = df_normalized[~outlier_mask].copy()
print(f"Final dataset: {df_final.shape[0]} images")
print(f"Final sample by group:")
print(df_final.groupby(['EthnicitySelf', 'GenderSelf']).size())

Found 6 extreme outliers
Final dataset: 460 images
Final sample by group:
EthnicitySelf  GenderSelf
A              F             46
               M             38
B              F             96
               M             78
L              F             22
               M             15
W              F             80
               M             85
dtype: int64


## Generate demographic contrast pairs

Three types: gender-only, ethnicity-only, intersectional contrasts

In [9]:
def calculate_covariate_distance(row1, row2, balance_vars_z):
    """Calculate Euclidean distance between two faces on balancing covariates."""
    vec1 = row1[balance_vars_z].values
    vec2 = row2[balance_vars_z].values
    return euclidean(vec1, vec2)

def generate_pairs_for_contrast(df, group1_filter, group2_filter, contrast_name, 
                               balance_vars_z, distance_threshold=1.5, max_pairs_per_face=2):
    """Generate balanced pairs for a demographic contrast."""
    group1 = df[group1_filter].copy()
    group2 = df[group2_filter].copy()
    
    if len(group1) == 0 or len(group2) == 0:
        print(f"Warning: {contrast_name} has empty groups")
        return []
    
    pairs = []
    face_usage = {}
    
    for _, face1 in group1.iterrows():
        for _, face2 in group2.iterrows():
            
            if (face_usage.get(face1['Model'], 0) >= max_pairs_per_face or 
                face_usage.get(face2['Model'], 0) >= max_pairs_per_face):
                continue
            
            distance = calculate_covariate_distance(face1, face2, balance_vars_z)
            
            if distance <= distance_threshold:
                pair_info = {
                    'contrast_type': contrast_name,
                    'left_model': face1['Model'],
                    'right_model': face2['Model'],
                    'left_ethnicity': face1['EthnicitySelf'],
                    'left_gender': face1['GenderSelf'],
                    'right_ethnicity': face2['EthnicitySelf'],
                    'right_gender': face2['GenderSelf'],
                    'covariate_distance': distance,
                    'pair_id': f"{contrast_name}_{face1['Model']}_{face2['Model']}"
                }
                
                # Add covariate values
                for var in balance_vars_z:
                    pair_info[f'left_{var}'] = face1[var]
                    pair_info[f'right_{var}'] = face2[var]
                
                pairs.append(pair_info)
                face_usage[face1['Model']] = face_usage.get(face1['Model'], 0) + 1
                face_usage[face2['Model']] = face_usage.get(face2['Model'], 0) + 1
    
    print(f"{contrast_name}: {len(pairs)} pairs")
    return pairs

## Generate all contrast pairs with reasonable distance threshold

In [10]:
# Generate all target contrasts
all_pairs = []

# 1. Gender-only contrasts (within ethnicity)
print("=== GENDER-ONLY CONTRASTS ===")
for ethnicity in ['A', 'B', 'L', 'W']:
    female_filter = (df_final['EthnicitySelf'] == ethnicity) & (df_final['GenderSelf'] == 'F')
    male_filter = (df_final['EthnicitySelf'] == ethnicity) & (df_final['GenderSelf'] == 'M')
    
    pairs = generate_pairs_for_contrast(
        df_final, female_filter, male_filter, 
        f'GenderOnly_{ethnicity}', balance_vars_z
    )
    all_pairs.extend(pairs)

# 2. Ethnicity-only contrasts (within gender)
print("\n=== ETHNICITY-ONLY CONTRASTS ===")
ethnicity_combinations = [('A', 'B'), ('A', 'L'), ('A', 'W'), ('B', 'L'), ('B', 'W'), ('L', 'W')]

for gender in ['F', 'M']:
    for eth1, eth2 in ethnicity_combinations:
        filter1 = (df_final['EthnicitySelf'] == eth1) & (df_final['GenderSelf'] == gender)
        filter2 = (df_final['EthnicitySelf'] == eth2) & (df_final['GenderSelf'] == gender)
        
        pairs = generate_pairs_for_contrast(
            df_final, filter1, filter2, 
            f'EthnicityOnly_{gender}_{eth1}vs{eth2}', balance_vars_z
        )
        all_pairs.extend(pairs)

# 3. Intersectional contrasts
print("\n=== INTERSECTIONAL CONTRASTS ===")
intersectional_contrasts = [
    (('F', 'W'), ('M', 'B')),  # White Female vs Black Male
    (('F', 'B'), ('M', 'W')),  # Black Female vs White Male
    (('F', 'A'), ('M', 'W')),  # Asian Female vs White Male
    (('F', 'W'), ('M', 'A')),  # White Female vs Asian Male
    (('F', 'L'), ('M', 'W')),  # Latino Female vs White Male
    (('F', 'W'), ('M', 'L')),  # White Female vs Latino Male
]

for (g1, e1), (g2, e2) in intersectional_contrasts:
    filter1 = (df_final['GenderSelf'] == g1) & (df_final['EthnicitySelf'] == e1)
    filter2 = (df_final['GenderSelf'] == g2) & (df_final['EthnicitySelf'] == e2)
    
    pairs = generate_pairs_for_contrast(
        df_final, filter1, filter2, 
        f'Intersectional_{g1}{e1}vs{g2}{e2}', balance_vars_z
    )
    all_pairs.extend(pairs)

print(f"\nTotal pairs generated: {len(all_pairs)}")

=== GENDER-ONLY CONTRASTS ===
GenderOnly_A: 48 pairs
GenderOnly_A: 48 pairs
GenderOnly_B: 124 pairs
GenderOnly_L: 21 pairs
GenderOnly_B: 124 pairs
GenderOnly_L: 21 pairs
GenderOnly_W: 119 pairs

=== ETHNICITY-ONLY CONTRASTS ===
GenderOnly_W: 119 pairs

=== ETHNICITY-ONLY CONTRASTS ===
EthnicityOnly_F_AvsB: 86 pairs
EthnicityOnly_F_AvsL: 39 pairs
EthnicityOnly_F_AvsB: 86 pairs
EthnicityOnly_F_AvsL: 39 pairs
EthnicityOnly_F_AvsW: 91 pairs
EthnicityOnly_F_BvsL: 44 pairs
EthnicityOnly_F_AvsW: 91 pairs
EthnicityOnly_F_BvsL: 44 pairs
EthnicityOnly_F_BvsW: 140 pairs
EthnicityOnly_F_LvsW: 43 pairs
EthnicityOnly_F_BvsW: 140 pairs
EthnicityOnly_F_LvsW: 43 pairs
EthnicityOnly_M_AvsB: 66 pairs
EthnicityOnly_M_AvsL: 27 pairs
EthnicityOnly_M_AvsB: 66 pairs
EthnicityOnly_M_AvsL: 27 pairs
EthnicityOnly_M_AvsW: 70 pairs
EthnicityOnly_M_BvsL: 29 pairs
EthnicityOnly_M_AvsW: 70 pairs
EthnicityOnly_M_BvsL: 29 pairs
EthnicityOnly_M_BvsW: 118 pairs
EthnicityOnly_M_LvsW: 30 pairs

=== INTERSECTIONAL CONTRASTS

## Sample 64 target pairs + 4 attention checks

In [11]:
# Convert pairs to DataFrame
pairs_df = pd.DataFrame(all_pairs)

def sample_balanced_pairs(pairs_df, target_total=64, random_seed=42):
    """Sample balanced set of 64 target pairs ensuring no identity repeats."""
    np.random.seed(random_seed)
    
    selected_pairs = []
    used_models = set()
    
    # 1. Gender-only pairs (aim for 16)
    gender_pairs = pairs_df[pairs_df['contrast_type'].str.contains('GenderOnly')].copy()
    gender_selected = []
    
    for ethnicity in ['A', 'B', 'L', 'W']:
        eth_pairs = gender_pairs[gender_pairs['contrast_type'] == f'GenderOnly_{ethnicity}']
        if len(eth_pairs) > 0:
            n_sample = min(4, len(eth_pairs))
            sampled = eth_pairs.sample(n=n_sample, random_state=random_seed+ord(ethnicity))
            gender_selected.extend(sampled.to_dict('records'))
            
            for _, row in sampled.iterrows():
                used_models.add(row['left_model'])
                used_models.add(row['right_model'])
    
    selected_pairs.extend(gender_selected[:16])
    print(f"Selected {len(gender_selected[:16])} gender-only pairs")
    
    # 2. Ethnicity-only pairs (aim for 24)
    ethnicity_pairs = pairs_df[pairs_df['contrast_type'].str.contains('EthnicityOnly')].copy()
    ethnicity_pairs = ethnicity_pairs[
        ~(ethnicity_pairs['left_model'].isin(used_models) | 
          ethnicity_pairs['right_model'].isin(used_models))
    ]
    
    if len(ethnicity_pairs) > 0:
        n_sample = min(24, len(ethnicity_pairs))
        sampled = ethnicity_pairs.sample(n=n_sample, random_state=random_seed+1)
        selected_pairs.extend(sampled.to_dict('records'))
        
        for _, row in sampled.iterrows():
            used_models.add(row['left_model'])
            used_models.add(row['right_model'])
        print(f"Selected {n_sample} ethnicity-only pairs")
    
    # 3. Intersectional pairs (fill remaining slots)
    intersectional_pairs = pairs_df[pairs_df['contrast_type'].str.contains('Intersectional')].copy()
    intersectional_pairs = intersectional_pairs[
        ~(intersectional_pairs['left_model'].isin(used_models) | 
          intersectional_pairs['right_model'].isin(used_models))
    ]
    
    if len(intersectional_pairs) > 0:
        remaining_needed = target_total - len(selected_pairs)
        n_sample = min(remaining_needed, len(intersectional_pairs))
        sampled = intersectional_pairs.sample(n=n_sample, random_state=random_seed+2)
        selected_pairs.extend(sampled.to_dict('records'))
        print(f"Selected {n_sample} intersectional pairs")
    
    return pd.DataFrame(selected_pairs)

# Sample 64 target pairs
target_pairs = sample_balanced_pairs(pairs_df, target_total=64)
print(f"\nFinal target pairs: {len(target_pairs)}")
print(target_pairs['contrast_type'].value_counts())

Selected 16 gender-only pairs
Selected 24 ethnicity-only pairs
Selected 24 intersectional pairs

Final target pairs: 64
contrast_type
Intersectional_FWvsMB    9
EthnicityOnly_F_BvsW     7
Intersectional_FBvsMW    6
EthnicityOnly_M_BvsW     5
GenderOnly_A             4
GenderOnly_B             4
GenderOnly_L             4
GenderOnly_W             4
EthnicityOnly_F_AvsW     4
Intersectional_FAvsMW    4
EthnicityOnly_F_AvsB     3
Intersectional_FLvsMW    3
EthnicityOnly_F_LvsW     2
EthnicityOnly_M_AvsW     2
Intersectional_FWvsMA    2
EthnicityOnly_M_LvsW     1
Name: count, dtype: int64


In [12]:
# Create 4 attention check trials
def create_attention_checks(pairs_df, n_checks=4):
    """Create attention check trials with clear instructions."""
    attention_pairs = pairs_df.sample(n_checks, random_state=123)
    attention_checks = []
    
    for i, (_, pair) in enumerate(attention_pairs.iterrows()):
        check = {
            'trial_type': 'attention_check',
            'trial_id': f'attention_{i+1}',
            'contrast_type': 'attention_check',
            'left_model': pair['left_model'],
            'right_model': pair['right_model'],
            'left_ethnicity': pair['left_ethnicity'],
            'left_gender': pair['left_gender'],
            'right_ethnicity': pair['right_ethnicity'],
            'right_gender': pair['right_gender'],
            'instruction': f'Select the face on the {"left" if i % 2 == 0 else "right"}',
            'correct_choice': 'left' if i % 2 == 0 else 'right'
        }
        attention_checks.append(check)
    
    return attention_checks

attention_trials = create_attention_checks(pairs_df, n_checks=4)
print(f"Created {len(attention_trials)} attention check trials")

Created 4 attention check trials


## Create final trial structure with side randomization

In [13]:
def create_final_trial_dataframe(target_pairs, attention_trials, random_seed=42):
    """Create final trial dataframe with side randomization."""
    np.random.seed(random_seed)
    
    all_trials = []
    
    # Add target trials
    for _, row in target_pairs.iterrows():
        trial = {
            'trial_id': f"target_{len(all_trials)+1:02d}",
            'trial_type': 'target',
            'contrast_type': row['contrast_type'],
            'left_model': row['left_model'],
            'right_model': row['right_model'],
            'left_ethnicity': row['left_ethnicity'],
            'left_gender': row['left_gender'],
            'right_ethnicity': row['right_ethnicity'],
            'right_gender': row['right_gender'],
            'covariate_distance': row['covariate_distance'],
            'is_attention_check': False
        }
        
        # Add original covariate values
        for var in balance_vars:
            trial[f'left_{var}'] = df_final[df_final['Model'] == row['left_model']][var].iloc[0]
            trial[f'right_{var}'] = df_final[df_final['Model'] == row['right_model']][var].iloc[0]
        
        all_trials.append(trial)
    
    # Add attention checks
    for trial in attention_trials:
        trial_dict = {
            'trial_id': trial['trial_id'],
            'trial_type': 'attention_check',
            'contrast_type': 'attention_check',
            'left_model': trial['left_model'],
            'right_model': trial['right_model'],
            'left_ethnicity': trial['left_ethnicity'],
            'left_gender': trial['left_gender'],
            'right_ethnicity': trial['right_ethnicity'],
            'right_gender': trial['right_gender'],
            'covariate_distance': np.nan,
            'is_attention_check': True,
            'correct_choice': trial['correct_choice']
        }
        
        # Add covariate values for attention checks too
        for var in balance_vars:
            trial_dict[f'left_{var}'] = df_final[df_final['Model'] == trial['left_model']][var].iloc[0]
            trial_dict[f'right_{var}'] = df_final[df_final['Model'] == trial['right_model']][var].iloc[0]
            
        all_trials.append(trial_dict)
    
    trials_df = pd.DataFrame(all_trials)
    
    # Add side randomization (50/50)
    trials_df['side_randomized'] = np.random.choice(['original', 'flipped'], 
                                                   size=len(trials_df), p=[0.5, 0.5])
    
    # Create display columns (after randomization)
    trials_df['display_left_model'] = trials_df['left_model'].copy()
    trials_df['display_right_model'] = trials_df['right_model'].copy()
    
    # Flip when randomized
    flipped_mask = trials_df['side_randomized'] == 'flipped'
    trials_df.loc[flipped_mask, 'display_left_model'] = trials_df.loc[flipped_mask, 'right_model']
    trials_df.loc[flipped_mask, 'display_right_model'] = trials_df.loc[flipped_mask, 'left_model']
    
    # Add presentation order
    trials_df['presentation_order'] = np.random.permutation(len(trials_df)) + 1
    trials_df = trials_df.sort_values('presentation_order').reset_index(drop=True)
    
    return trials_df

final_trials = create_final_trial_dataframe(target_pairs, attention_trials)
print(f"Final trial structure: {len(final_trials)} trials")
print(f"Target: {sum(final_trials['trial_type'] == 'target')}")
print(f"Attention: {sum(final_trials['is_attention_check'])}")

Final trial structure: 68 trials
Target: 64
Attention: 4


## Map to image files and create balance table

In [14]:
# Map model IDs to JPG filenames
import os
import glob

def create_model_to_filename_mapping(neutral_faces_path):
    jpg_files = glob.glob(os.path.join(neutral_faces_path, "*.jpg"))
    model_to_filename = {}
    
    for filepath in jpg_files:
        filename = os.path.basename(filepath)
        if filename.startswith('CFD-') and filename.endswith('-N.jpg'):
            parts = filename.split('-')
            if len(parts) >= 4:
                ethnicity_gender = parts[1]  # e.g., 'AF'
                number = parts[2]            # e.g., '200'
                model_id = f"{ethnicity_gender}-{number}"  # e.g., 'AF-200'
                model_to_filename[model_id] = filename
    
    return model_to_filename

model_to_filename = create_model_to_filename_mapping("../neutral_faces")
print(f"Mapped {len(model_to_filename)} models to filenames")

# Add filename columns
final_trials['left_image_file'] = final_trials['display_left_model'].map(model_to_filename)
final_trials['right_image_file'] = final_trials['display_right_model'].map(model_to_filename)

missing = final_trials['left_image_file'].isnull().sum() + final_trials['right_image_file'].isnull().sum()
print(f"Missing image files: {missing}")
if missing == 0:
    print("✅ All trials mapped to image files!")

Mapped 597 models to filenames
Missing image files: 0
✅ All trials mapped to image files!


In [15]:
# Create balance table showing left vs right image characteristics
balance_data = []

# Collect data for left images
left_models = final_trials['display_left_model'].tolist()
left_data = df_final[df_final['Model'].isin(left_models)]

# Collect data for right images  
right_models = final_trials['display_right_model'].tolist()
right_data = df_final[df_final['Model'].isin(right_models)]

# Calculate means for each covariate
balance_vars_all = ['AgeRated', 'Attractive', 'Dominant', 'Trustworthy']

print("=== BALANCE TABLE: Left vs Right Images ===")
print("Should see minimal differences if randomization worked well\n")

balance_table = pd.DataFrame({
    'Covariate': balance_vars_all + ['EthnicitySelf_A', 'EthnicitySelf_B', 'EthnicitySelf_L', 'EthnicitySelf_W', 'GenderSelf_F', 'GenderSelf_M'],
    'Left_Images_Mean': [
        left_data['AgeRated'].mean(),
        left_data['Attractive'].mean(), 
        left_data['Dominant'].mean(),
        left_data['Trustworthy'].mean(),
        (left_data['EthnicitySelf'] == 'A').mean(),
        (left_data['EthnicitySelf'] == 'B').mean(),
        (left_data['EthnicitySelf'] == 'L').mean(),
        (left_data['EthnicitySelf'] == 'W').mean(),
        (left_data['GenderSelf'] == 'F').mean(),
        (left_data['GenderSelf'] == 'M').mean()
    ],
    'Right_Images_Mean': [
        right_data['AgeRated'].mean(),
        right_data['Attractive'].mean(),
        right_data['Dominant'].mean(), 
        right_data['Trustworthy'].mean(),
        (right_data['EthnicitySelf'] == 'A').mean(),
        (right_data['EthnicitySelf'] == 'B').mean(),
        (right_data['EthnicitySelf'] == 'L').mean(),
        (right_data['EthnicitySelf'] == 'W').mean(),
        (right_data['GenderSelf'] == 'F').mean(),
        (right_data['GenderSelf'] == 'M').mean()
    ]
})

balance_table['Difference'] = balance_table['Right_Images_Mean'] - balance_table['Left_Images_Mean']
balance_table['Abs_Difference'] = abs(balance_table['Difference'])

print(balance_table.round(3))

print(f"\n=== BALANCE ASSESSMENT ===")
print(f"Mean absolute difference across all covariates: {balance_table['Abs_Difference'].mean():.3f}")
print(f"Max absolute difference: {balance_table['Abs_Difference'].max():.3f}")

# Check if differences are small (good balance)
large_diffs = balance_table[balance_table['Abs_Difference'] > 0.1]
if len(large_diffs) == 0:
    print("✅ Good balance achieved - all differences < 0.1")
else:
    print(f"⚠️  Large differences found in:")
    for _, row in large_diffs.iterrows():
        print(f"   {row['Covariate']}: {row['Difference']:.3f}")
        
print(f"\nNote: Differences should be small if side randomization worked well.")
print("Large differences suggest systematic bias in left/right assignment.")

=== BALANCE TABLE: Left vs Right Images ===
Should see minimal differences if randomization worked well

         Covariate  Left_Images_Mean  Right_Images_Mean  Difference  \
0         AgeRated            27.815             28.064       0.249   
1       Attractive             3.206              3.296       0.090   
2         Dominant             2.706              2.752       0.046   
3      Trustworthy             3.488              3.477      -0.012   
4  EthnicitySelf_A             0.164              0.231       0.067   
5  EthnicitySelf_B             0.373              0.215      -0.158   
6  EthnicitySelf_L             0.134              0.062      -0.073   
7  EthnicitySelf_W             0.328              0.492       0.164   
8     GenderSelf_F             0.582              0.538      -0.044   
9     GenderSelf_M             0.418              0.462       0.044   

   Abs_Difference  
0           0.249  
1           0.090  
2           0.046  
3           0.012  
4           0

## Final 68 trials ready for survey

In [16]:
# Display final trial structure
print("=== FINAL TRIAL SUMMARY ===")
print(f"Total trials: {len(final_trials)}")
print(f"Target trials: {sum(final_trials['trial_type'] == 'target')}")
print(f"Attention checks: {sum(final_trials['is_attention_check'])}")

# Show contrast distribution
target_only = final_trials[final_trials['trial_type'] == 'target']
print(f"\nContrast type distribution:")
contrast_counts = target_only['contrast_type'].value_counts()
for contrast_type, count in contrast_counts.items():
    print(f"  {contrast_type}: {count}")

# Show sample trials
print(f"\n=== SAMPLE TRIALS ===")
sample_cols = ['presentation_order', 'trial_type', 'contrast_type', 
               'left_image_file', 'right_image_file', 'side_randomized']
print(final_trials[sample_cols].head(10).to_string(index=False))

# Export instruction
print(f"\n=== EXPORT READY ===")
print("To save: final_trials.to_csv('pictopercept_68_trials.csv', index=False)")
print("Ready for Qualtrics implementation ✅")

=== FINAL TRIAL SUMMARY ===
Total trials: 68
Target trials: 64
Attention checks: 4

Contrast type distribution:
  Intersectional_FWvsMB: 9
  EthnicityOnly_F_BvsW: 7
  Intersectional_FBvsMW: 6
  EthnicityOnly_M_BvsW: 5
  GenderOnly_B: 4
  Intersectional_FAvsMW: 4
  GenderOnly_L: 4
  GenderOnly_W: 4
  EthnicityOnly_F_AvsW: 4
  GenderOnly_A: 4
  EthnicityOnly_F_AvsB: 3
  Intersectional_FLvsMW: 3
  EthnicityOnly_F_LvsW: 2
  EthnicityOnly_M_AvsW: 2
  Intersectional_FWvsMA: 2
  EthnicityOnly_M_LvsW: 1

=== SAMPLE TRIALS ===
 presentation_order      trial_type         contrast_type      left_image_file     right_image_file side_randomized
                  1 attention_check       attention_check CFD-BM-013-002-N.jpg CFD-BF-003-003-N.jpg         flipped
                  2          target          GenderOnly_B CFD-BF-008-001-N.jpg CFD-BM-001-014-N.jpg        original
                  3          target  EthnicityOnly_F_BvsW CFD-BF-010-001-N.jpg CFD-WF-013-003-N.jpg        original
            